**Imports**

In [11]:
%matplotlib ipympl
from ipywidgets import interact, interactive, ToggleButton, IntSlider, FloatSlider, Output
import ipywidgets as widgets
from matplotlib import pyplot as plt
from wingstructure import data, aero

# Airfoil definition

In [12]:
# create wing object
wing = data.Wing()

# add sections to wing
wing.add_section(data.Point(0.0, 0.0, 0.0), 1.0, 0.0)
wing.add_section(data.Point(0.05, 4.25, 0.0), 0.7, 0.0)
wing.add_section(data.Point(0.1, 7.75, 0.0), 0.35, 0.0)

# set fuselage with (=root of wing) to zero
wing.set_root_pos(0.0)

# define spoiler position
wing.set_spoiler(1.5, 2.9)

# define control-surfaces
wing.set_flap('flap', 1, 2.8,[0.7,0.7])
wing.set_flap('flap2', 4.25, 7, [0.7,0.8])

# display simple wing
plt.figure(figsize=(8,5))
wing.plot()

FigureCanvasNbAgg()

# Lift calculation

In [13]:
class liftplotter:
    def __init__(self):
        
        self.liftana = aero.LiftAnalysis(wing=wing)
        
        self.fig = plt.figure()
        
        self.ax = plt.gca()
        
        plt.ylabel('lokaler Auftriebsbeiwert $c_l$')
        plt.xlabel('Spannweitenposition y in m')
        
        plt.ylim(-1.7, 2.1)
        
        res = self._calc(0.0)
        plt.grid()
        
        self.line = self.ax.plot(self.liftana.calc_ys, res[1])[0]
        
        self.text = plt.text(8, -1.7, r'$\alpha = {}^\circ$'.format(res[0]),
                             horizontalalignment='right')
    
    def _calc(self, C_L, aileron_l=0.0, aileron_r=0.0, spoiler=False):
        
        flap = {'flap2': [aileron_l, aileron_r]}
    
        res = self.liftana.calculate(C_L, air_brake=spoiler, flap_deflections=flap)
        
        return res
    
    def plotdistri(self,C_L, spoiler=False, aileron_l=0.0, aileron_r=0.0):
        
        res = self._calc(C_L, spoiler, aileron_l, aileron_r)
        
        self.line.set_ydata(res[1])
        self.text.set_text(r'$\alpha = {:.2f}^\circ$'.format(res[0]))

In [14]:
lp = liftplotter()

def pltfun(C_L, aileron_l, aileron_r, spoiler):
    
    lp.plotdistri(C_L, aileron_l, aileron_r, spoiler)

lift_slider = FloatSlider(value=0.0, min=-1.0, max=1.5)
spoiler_button = ToggleButton(value=False, description='spoiler')
aileron_slider_l = FloatSlider(value=0.0, min=-8, max=9)
aileron_slider_r = FloatSlider(value=0.0, min=-8, max=9)
    
interact(pltfun, 
         C_L=lift_slider, 
         aileron_l=aileron_slider_l,
         aileron_r=aileron_slider_r,
         spoiler=spoiler_button);

/home/jonathan/Programmieren/wingstructure/wingstructure/aero/analysis.py:33: UserWarning: No airfoil database defined, using default airfoil.
  warn('No airfoil database defined, using default airfoil.')


FigureCanvasNbAgg()

interactive(children=(FloatSlider(value=0.0, description='C_L', max=1.5, min=-1.0), FloatSlider(value=0.0, des…

In [15]:
from wingstructure import structure
import collections as col

# Material Definition
Material = col.namedtuple('Material', ['ρ'])

fabric = Material(ρ=0.3e3)
foam = Material(ρ=0.18e3)

# load airfoil.dat file
coords = np.loadtxt('airfoils/ah93157.dat', skiprows=1)*1.2

# initialize base for section analysis
secbase = structure.SectionBase(coords)

# add shell layers
out = structure.Layer(fabric, 2e-3)
core = structure.Layer(fabric, 5e-3)
inner = structure.Layer(fabric, 2e-3)
secbase.append(out)
secbase.append(core)
secbase.append(inner)

# add spar
spar = structure.ISpar({'flange': fabric, 'web': fabric}, 0.5, 0.3, 3e-2, 0.8, 5e-3)
secbase.append(spar)

w = widgets.Output()

ma = structure.MassAnalysis(secbase)

display(w)

class secshow:
    def __init__(self, secbase, ma):
        self.w = widgets.Output()
        self.secbase = secbase
        self.ma = ma
        
    def showding(self,t, t2, t3, p, p2):
        self.secbase.features[1].thickness=t2*1e-3
        self.secbase.features[3].flangethickness = t*1e-3
        self.secbase.features[3].webthickness = t3*1e-3
        self.secbase.features[3].midpos = p
        self.secbase.features[3].webpos = p2
        massprops = self.ma.massproperties
        with w:
            w.clear_output(wait=True)
            display(self.secbase)
            display('Masse/Länge: {:4f} kg/m'.format(massprops[1]))

showding = secshow(secbase, ma)

shellthickness_slider = FloatSlider(value=1, min=0.5, max=25, step=2.5, description='Balsadicke')
flangethickness_slider = IntSlider(value=20, min=10, max=35, description='Holmdicke')
webthickness_slider = IntSlider(value=3, min=1, max=15, description='Stegbreite')
sparpos_slider = FloatSlider(value=0.5, min=0.3, max=0.7, step=0.05, description='Holmposition')
webpos_slider = FloatSlider(value=0.5, min=0.0, max=1.0, step=0.05, description='Stegposition')

interact(showding.showding, t=flangethickness_slider, t2=shellthickness_slider, 
         t3=webthickness_slider, p=sparpos_slider, p2=webpos_slider);

Output()

interactive(children=(IntSlider(value=20, description='Holmdicke', max=35, min=10), FloatSlider(value=1.0, des…

In [5]:
import numpy as np
from wingstructure.structure import beammechanics as beam
%matplotlib ipympl
from matplotlib import pyplot as plt

# creating test geometry

geometry_section = np.load('contour.npy')

youngsmoduli_left = np.ones(len(geometry_section[0])-1)*12.5*10**9
youngsmoduli_middle = np.ones(len(geometry_section[1])-1)*12.5*10*9
youngsmoduli_right = np.ones(len(geometry_section[2])-1)*12.5*10**9

shearmoduli_left = np.ones(len(geometry_section[0])-1)*19.6*10**9
shearmoduli_middle = np.ones(len(geometry_section[1])-1)*19.6*10**9
shearmoduli_right = np.ones(len(geometry_section[2])-1)*19.6*10**9

thicknesses_left = np.ones(len(geometry_section[0])-1)*0.003
thicknesses_middle = np.ones(len(geometry_section[1])-1)*0.003
thicknesses_right = np.ones(len(geometry_section[2])-1)*0.003

youngsmoduli_section = youngsmoduli_left, youngsmoduli_middle, youngsmoduli_right
shearmoduli_section = shearmoduli_left, shearmoduli_middle, shearmoduli_right
thicknesses_section = thicknesses_left, thicknesses_middle, thicknesses_right

generate_testsection = lambda : beam.Crosssection(geometry_section, youngsmoduli_section, shearmoduli_section, thicknesses_section)
testsections = [generate_testsection()]*60

geol = testsections[0].geometries[0]
geom = testsections[0].geometries[1]
geor = testsections[0].geometries[2]


def plotdistri(Qy, Qz, T):
    
    Q = [Qy,Qz]
    Q_new = beam.transform(np.array((Q)), testsections[0].Θ)
    shearflow = testsections[0].shearflow_total(Q_new, T)
    shearflow = np.array(shearflow)/30
    
    plt.clf()
    
    shearflow_plot = testsections[0].plot_around_profil(shearflow)

    shearflow_plot_left = shearflow_plot[0]
    shearflow_plot_middle = shearflow_plot[1]
    shearflow_plot_right = shearflow_plot[2]
    
    center = testsections[0].shearcenter()
    t4 = testsections[0].t4_point()
    
    plt.plot(geol[:,0], geol[:,1])
    plt.plot(geom[:,0], geom[:,1])
    plt.plot(geor[:,0], geor[:,1])
    plt.plot(center[0], center[1], 'o')
    plt.plot(t4[0], t4[1], 'x')
    plt.axis('equal')
    
    
    plt.plot(shearflow_plot_left[:,0], shearflow_plot_left[:,1], 'orange')
    plt.plot(shearflow_plot_middle[:,0], shearflow_plot_middle[:,1], 'orange')
    plt.plot(shearflow_plot_right[:,0], shearflow_plot_right[:,1], 'orange')

    plt.arrow(t4[0], t4[1], 0, -Qz/10, linewidth=2, color='red')
    plt.arrow(t4[0], t4[1], -Qy/10, 0, linewidth=2, color='red')



    line1 = np.array((shearflow_plot_left[0], shearflow_plot_right[0]))
    plt.plot(line1[:,0], line1[:,1], 'orange')

    line2 = np.array((shearflow_plot_left[-1], shearflow_plot_right[-1]))
    plt.plot(line2[:,0], line2[:,1], 'orange')
    
    

from ipywidgets import interact, interactive, fixed, interact_manual, FloatSlider
import ipywidgets as widgets

plt.figure()


Q_y_slider = FloatSlider(value=0.0, min=-5, max=5)
Q_z_slider = FloatSlider(value=0.0, min=-5, max=5)
T_slider = FloatSlider(value=0.0, min=-5, max=5)

interact(plotdistri, Qy=Q_y_slider, Qz=Q_z_slider, T=T_slider)

FigureCanvasNbAgg()

interactive(children=(FloatSlider(value=0.0, description='Qy', max=5.0, min=-5.0), FloatSlider(value=0.0, desc…

<function __main__.plotdistri(Qy, Qz, T)>